In [1]:
import os
import pandas as pd
import numpy as np

In [2]:
df1 = pd.read_excel('TP_pivot.xlsx', sheet_name='TODO')

In [3]:
df2 = df1.loc[df1['Tipo de tienda'] != 'CONVENIENCIA']

In [4]:
df3 = df2[['Store Number','SEMANA','City','State','Tipo de tienda','Region','Retailer',
'CADENA','ruta','Area Manager','Area Executive','Whisky','Tequila','Liqueur','Gin','Vodka']]

In [5]:
weeks = [40,41,42,43,44]

In [47]:
weeks_headers = ['week40','week41','week42','week43','week44']

In [6]:
df4 = df3[df3['SEMANA'].isin(weeks)]

In [7]:
df5 = df4.groupby(['Store Number','SEMANA','City','State','Tipo de tienda','Region','Retailer','CADENA','ruta',
'Area Manager','Area Executive']).sum().reset_index()

In [8]:
df5['Tequila Score'] = np.where((df5['Whisky'] >= 2) & (df5['Tequila'] >= 1), (df5['Tequila'])*2, 0)

In [9]:
df5['Liqueur Score'] = np.where((df5['Whisky'] >= 2) & (df5['Liqueur'] >= 1), (df5['Liqueur'])*1, 0)

In [10]:
df5['Gin Score'] = np.where((df5['Whisky'] >= 2) & (df5['Gin'] >= 1), (df5['Gin'])*1, 0)

In [11]:
df5['Vodka Score'] = np.where((df5['Whisky'] >= 2) & (df5['Vodka'] >= 1), (df5['Vodka'])*1, 0)

In [12]:
df5['Total Score'] = df5['Tequila Score'] + df5['Liqueur Score'] + df5['Gin Score'] + df5['Vodka Score']

In [13]:
def conditionA(row):
    if row['Tequila'] >= 1:
        return 1
    else:
        return 0

In [14]:
def conditionB(row):
    if row['Liqueur'] >= 1:
        return 1
    else:
        return 0

In [15]:
def conditionC(row):
    if row['Gin'] >= 1:
        return 1
    else:
        return 0

In [16]:
def conditionD(row):
    if row['Vodka'] >= 1:
        return 1
    else:
        return 0

In [17]:
df5['Tequila2'] = df5.apply(lambda row: conditionA(row), axis=1)

In [18]:
df5['Liqueur2'] = df5.apply(lambda row: conditionB(row), axis=1)

In [19]:
df5['Gin2'] = df5.apply(lambda row: conditionC(row), axis=1)

In [20]:
df5['Vodka2'] = df5.apply(lambda row: conditionD(row), axis=1)

In [21]:
df5['Tequila2'] = df5['Tequila2'] / 5
df5['Liqueur2'] = df5['Liqueur2'] / 5
df5['Gin2'] = df5['Gin2'] / 5
df5['Vodka2'] = df5['Vodka2'] / 5

In [89]:
df_score = df5[['Store Number','ruta','SEMANA',
'Whisky','Tequila','Liqueur','Gin','Vodka','Total Score']]

In [23]:
df_extrap = df5[['Store Number','ruta','SEMANA',
'Tequila2','Liqueur2','Gin2','Vodka2']]

In [24]:
df_score_ruta = df_score.groupby(['ruta','SEMANA']).agg({
'Store Number': 'count',
'Whisky': 'sum',
'Tequila': 'sum',
'Liqueur': 'sum',
'Gin': 'sum',
'Vodka': 'sum',
'Total Score': 'sum',
}).reset_index()

In [25]:
df_score_ruta = df_score_ruta[['ruta','SEMANA','Total Score']]

In [48]:
pivot_weekly_score = pd.pivot_table(df_score_ruta,
index=['ruta'],
columns=['SEMANA'], aggfunc='sum', fill_value=0).reset_index()

In [49]:
pivot_weekly_score.columns = pivot_weekly_score.columns.get_level_values(0)

In [50]:
pivot_weekly_score.columns = pivot_weekly_score.columns[:1].tolist() + weeks_headers

In [51]:
df_extrap_store = df_extrap.groupby(['Store Number','ruta']).agg({
'Store Number': 'count',
'Tequila2': 'sum',
'Liqueur2': 'sum',
'Gin2': 'sum',
'Vodka2': 'sum',
})

In [52]:
df_extrap_store = df_extrap_store.rename(columns={'Store Number': 'Weeks visited'}).reset_index()

In [53]:
df_extrap_store['Tequila EP'] = [2 if x == 1 else 0 for x in df_extrap_store['Tequila2']]
df_extrap_store['Liqueur EP'] = [2 if x == 1 else 0 for x in df_extrap_store['Liqueur2']]
df_extrap_store['Gin EP'] = [2 if x == 1 else 0 for x in df_extrap_store['Gin2']]
df_extrap_store['Vodka EP'] = [2 if x == 1 else 0 for x in df_extrap_store['Vodka2']]
df_extrap_store['Total EP'] = df_extrap_store['Tequila EP'] + df_extrap_store['Liqueur EP'] + df_extrap_store['Gin EP'] + df_extrap_store['Vodka EP']

In [54]:
df_extrap_store = df_extrap_store[['Store Number','ruta','Weeks visited','Tequila EP','Liqueur EP','Gin EP','Vodka EP','Total EP']]

In [55]:
df_extrap_ruta = df_extrap_store[['ruta','Total EP']]

In [56]:
df_extrap_ruta = df_extrap_ruta.groupby('ruta').sum().reset_index()

In [62]:
df_leaderboard = pivot_weekly_score.merge(df_extrap_ruta, on='ruta')

In [63]:
df_leaderboard['Final Score'] = df_leaderboard.sum(axis=1, numeric_only=True)

In [64]:
df_leaderboard = df_leaderboard.sort_values(by='Final Score', ascending=False)

In [69]:
df_leaderboard['Lugar'] = np.arange(1,len(df_leaderboard)+1)

In [70]:
df_leaderboard.head() #portada del sitio web, ordenar del primer al ultimo lugar

,ruta,week40,week41,week42,week43,week44,Total EP,Final Score,Lugar
91,PROM.SEM.21,10,9,8,10,11,4,52,1
230,PROM.SEM.66,7,7,6,6,5,6,37,2
37,PROM.SEM.144,4,4,8,7,7,6,36,3
243,PROM.SEM.82,6,3,7,10,5,2,33,4
75,PROM.SEM.192,4,4,3,10,8,2,31,5


In [38]:
df_extrap_store.head() #usar esta en una pagina para comprobar puntos extra

,Store Number,ruta,Weeks visited,Tequila EP,Liqueur EP,Gin EP,Vodka EP,Total EP
0,3,PROM.SEM.153,3,0,0,0,0,0
1,5,PROM.SEM.153,1,0,0,0,0,0
2,6,PROM.SEM.153,2,0,0,0,0,0
3,23,PROM.SEM.316,5,0,0,0,2,2
4,26,PROM.SEM.335,5,0,0,0,0,0


In [102]:
df_score.head() #usar esta en una pagina para comprobar puntos base, con filtros en store, ruta y semana 

,store,ruta,week,whisky,tequila,licor,ginebra,vodka,score,name
0,3,PROM.SEM.153,41,1,0,0,0,0,0,3 MERZA TRADICIONAL ESTACION (SD)
1,3,PROM.SEM.153,42,1,0,0,0,0,0,3 MERZA TRADICIONAL ESTACION (SD)
2,3,PROM.SEM.153,43,2,0,0,0,0,0,3 MERZA TRADICIONAL ESTACION (SD)
3,5,PROM.SEM.153,43,2,0,0,0,0,0,5 MERZA TRADICIONAL BODEGA DE DESCUENTOS (SD)
4,6,PROM.SEM.153,42,1,0,0,0,0,0,6 MERZA TRADICIONAL JARDINADAS (SD)


In [91]:
df_score.loc[df_score['ruta'] == 'PROM.SEM.03']['Total Score'].sum()

17

In [60]:
df_leaderboard = df_leaderboard.rename(columns={
'Total EP': 'ep',
'Final Score': 'fs'
})

In [44]:
df_extrap_store = df_extrap_store.rename(columns={
'Store Number': 'store',
'SEMANA': 'week',
'Weeks visited': 'wv',
'Whisky EP': 'whisky',
'Tequila EP': 'tequila',
'Liqueur EP': 'licor',
'Gin EP': 'ginebra',
'Vodka EP': 'vodka',
'Total EP': 'ep'
})

In [96]:
df_score = df_score.rename(columns={
'Store Number': 'store',
'SEMANA': 'week',
'Whisky': 'whisky',
'Tequila': 'tequila',
'Liqueur': 'licor',
'Gin': 'ginebra',
'Vodka': 'vodka',
'Total Score': 'score'
})

In [ ]:
df2_store_names = df2[['Store Number','Store Name']]

In [ ]:
df2_store_names = df2_store_names.rename(columns={
'Store Number': 'store',
'Store Name': 'name'
})

In [ ]:
df2_store_names = df2_store_names.drop_duplicates(subset='store')

In [ ]:
df_score = df_score.merge(df2_store_names, how='left')

In [104]:
df_score = df_score[['name','ruta','week','whisky','tequila','licor','ginebra','vodka','score']]

In [71]:
df_leaderboard.to_json('leaderboard.js', orient='records')

In [108]:
df_score.to_json('score.js', orient='records')

In [103]:
df_score.count() #trate el nombre del PDV, pero asegurate que se mantenga este conteo

store      4773
ruta       4773
week       4773
whisky     4773
tequila    4773
licor      4773
ginebra    4773
vodka      4773
score      4773
name       4773
dtype: int64

In [40]:
#haz lo mismo para conveniencia, toma en cuenta VDM = 1 WHISKY + 1 ADICIONAL